In [6]:
using TrajectoryOptimization
using LinearAlgebra

In [7]:
N = 200
n = 6
m = 3
tf = 50.0
dt = tf/N
x0 = [2000.0, 1500.0, 1000.0, 0.0, 0.0, 0.0]
xf = zeros(n)

params = Dict("num_iter" => 800, # number of iteration of ADMM
              "N" => N, # horizon
              "n"=> n, # state size
              "m"=> m, # control size
              "ρ" => 1e-2, # weighting parameter for augmented lagrangian
              "α" => 1.0, # weighting parameter 
              "dt" => dt, # time step
              "tf" => tf, # final time 
              "ν0" => ones(N, m), # initial dual      
              "x0" => x0, # initial state     
              "xf" => xf, # final state     
              "u_min" => -5.0, # lower bound on u                 
              "u_max" => 5.0, # upper bound on u   
              "Q" => 1.0 * Matrix{Float64}(I, n, n), # Quadratic stage cost for states (n,n)
              "R" => 1.0 * Matrix{Float64}(I, m, m), # Quadratic stage cost for controls (m,m)
              "H" => zeros(m, n), # Quadratic Cross-coupling for state and controls (m,n)
              "q" => zeros(n), # Linear term on states (n,)
              "r" => zeros(m), # Linear term on controls (m,)
              "c" => 0.0, # constant term
              "d" => 1.0, # constant term
              "Qf" => 1.0 * Matrix{Float64}(I, n, n), # Quadratic final cost for terminal state (n,n)
              "qf" => zeros(n), # Linear term on terminal state (n,)
              "cf" => 0.0, # constant term (terminal)
               )

Dict{String,Any} with 23 entries:
  "ν0"       => [1.0 1.0 1.0; 1.0 1.0 1.0; … ; 1.0 1.0 1.0; 1.0 1.0 1.0]
  "Q"        => [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0…
  "c"        => 0.0
  "x0"       => [2000.0, 1500.0, 1000.0, 0.0, 0.0, 0.0]
  "dt"       => 0.25
  "r"        => [0.0, 0.0, 0.0]
  "xf"       => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  "num_iter" => 800
  "N"        => 200
  "tf"       => 50.0
  "u_min"    => -5.0
  "q"        => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  "H"        => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
  "d"        => 1.0
  "Qf"       => [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0…
  "u_max"    => 5.0
  "α"        => 1.0
  "qf"       => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  "m"        => 3
  "cf"       => 0.0
  "ρ"        => 0.01
  "R"        => [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]
  "n"        => 6

In [8]:
function cw_dynamics!(ẋ, x, u, params)
    n_cst = sqrt(params["μ"]/params["a"]^3)
    ẋ[1] = x[4]
    ẋ[2] = x[5]
    ẋ[3] = x[6]
    ẋ[4] = 3*n_cst^2*x[1] + 2*n_cst*x[5] + u[1]
    ẋ[5] = -2*n_cst*x[4] + u[2]
    ẋ[6] = -n_cst^2*x[3] + u[3] 
end

function cw_dynamics!(ẋ,x,u)
    cw_dynamics!(ẋ, x, u, params)
end    

cw_dynamics! (generic function with 2 methods)

In [9]:
# Create a L1Cost
l1cost = TrajectoryOptimization.L1Cost(
    params["Q"],
    params["R"],
    params["H"],
    params["q"],
    params["r"],
    params["c"],
    params["d"],
    params["Qf"],
    params["qf"],
    params["cf"])

model = Model(cw_dynamics!, n, m)

# X = fill(fill(0.0, params["N"]), params["n"])
# U = fill(fill(0.0, params["N"]-1), params["m"])
# Lower control bounds (m,)
u_min = params["u_min"] * ones(params["m"])
# Upper control bounds (m,)
u_max = params["u_max"] * ones(params["m"])

unconstrained_objective = UnconstrainedObjective(l1cost, params["tf"], params["x0"], params["xf"])
constrained_objective = constrained_objective = ConstrainedObjective(unconstrained_objective, 
    u_min=u_min, u_max=u_max, use_xf_equality_constraint=false);
    
# l1solver = Solver(model, unconstrained_objective, N=N+1);
l1solver = Solver(model, constrained_objective, N=N+1);

U = zeros(N, m);
results, stats = solve(l1solver, convert(Array{Float64,2}, U));   

# l1problem = TrajectoryOptimization.Problem(model, l1cost, params["x0"], X,
#         U, params["N"], params["dt"], params["tf"])
# l1opts = TrajectoryOptimization.

# # Create a L1Problem
# unconstrained_objective = UnconstrainedObjective(l1cost, params["tf"], params["x0"], params["xf"])

┌ Info: Interpolating initial guess
└ @ TrajectoryOptimization /home/simon/.julia/dev/TrajectoryOptimization/src/ilqr_methods.jl:242
┌ Info: Solving Constrained Problem...
└ @ TrajectoryOptimization /home/simon/.julia/dev/TrajectoryOptimization/src/ilqr_methods.jl:285


KeyError: KeyError: key "μ" not found